In [ ]:
# RL - Q-Learning con Keras y Gymnasium
################################
# Este cuaderno Jupyter presenta una implementación del algoritmo  Q-Learning (DQN) aplicado en entornos de la librería Gymnasium. 
# Utilizando la librería Keras para construir y entrenar una red neuronal, exploramos cómo un agente aprender de las recompensas del entorno.
# Creado por: [@MrCabss69]
# Fecha de creación: Thu Mar 07 2024


In [ ]:
import gymnasium as gym
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
def create_model(input_shape, num_actions):
    """Construye y devuelve el modelo de red neuronal."""
    model = Sequential([
        Dense(512, activation='relu', input_shape=input_shape),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(num_actions, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

def get_action(state, model):
    """Devuelve la acción basada en la política derivada del modelo."""
    q_values = model.predict(state)
    return np.argmax(q_values[0])

def train_model(model, env, episodes=1):
    """Entrena el modelo en el entorno especificado para un número de episodios."""
    for episode in range(1, episodes + 1):
        state, _ = env.reset()
        state = np.expand_dims(state, axis=0)
        done = False
        score = 0
        
        while not done:
            action = get_action(state, model)
            next_state, reward, terminated, truncated, _ = env.step(action)
            next_state = np.expand_dims(next_state, axis=0)
            score += reward
            done = terminated or truncated

            # Preparar el target para el entrenamiento
            target = reward + 0.99 * np.amax(model.predict(next_state)[0]) if not done else reward
            target_f = model.predict(state)
            target_f[0][action] = target
            
            # Entrenar el modelo
            model.fit(state, target_f, epochs=1, verbose=0)
            state = next_state
            
        print(f"Episode: {episode}/{episodes}, Score: {score}")

In [ ]:
# Inicialización y configuración del entorno
env = gym.make("CartPole-v1")
observation, _ = env.reset()
num_actions = env.action_space.n

In [ ]:
# Inicialización del modelo
model = create_model(env.observation_space.shape, num_actions)

In [ ]:
# Entrenamiento del modelo
train_model(model, env, episodes=1000)

In [ ]:
model.save('/trained/ql_1k.h5')

In [ ]:
# Cierre del entorno
env.close()